In [1]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader # permet de télécharger un dataset HuggingFace
from langchain_community.embeddings import HuggingFaceEmbeddings # on récupère un embedding adapté aux données HuggingFace
from langchain.indexes.vectorstore import VectorstoreIndexCreator # permet à Langchain de transformer les données en vecteurs

In [2]:
dataset_name = "tweet_eval"
page_content_column = "text"
name = "stance_climate"

loader = HuggingFaceDatasetLoader(dataset_name, page_content_column, name)

In [4]:
embeddings = HuggingFaceEmbeddings()

index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])

c:\Users\jeans\Desktop\rag_databird\rag_env\lib\site-packages\langchain\indexes\vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(
c:\Users\jeans\Desktop\rag_databird\rag_env\lib\site-packages\datasets\load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
Generating validation split: 100%|██████████| 40/40 [00:00<00:00, 6129.11 examples/s]


In [5]:
print(type(index))

<class 'langchain.indexes.vectorstore.VectorStoreIndexWrapper'>
